In [1]:
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from tqdm import tqdm

In [2]:
def load_data(data_range, data_type):
    data_x = []
    data_y = []
    for i in range(*data_range):
        data_x.append(np.load(f"../../datasets/artifact_features/vgg/{data_type}_x_{i}.npy"))
        data_y.append(np.load(f"../../datasets/artifact_features/vgg/{data_type}_y_{i}.npy"))
    data_x = np.concatenate(np.array(data_x), axis=0)
    data_y = np.concatenate(np.array(data_y), axis=0)
    return data_x, data_y

In [3]:
# train_x, train_y = load_data((0, 2), "train")
valid_x, valid_y = load_data((0, 1), "valid")

In [4]:
print(f'Training x shape: {train_x.shape}\n'
      f'Training y shape: {train_y.shape}\n'
      f'Valid x shape: {valid_x.shape}\n'
      f'Valid y shape: {valid_y.shape}')

NameError: name 'train_x' is not defined

In [11]:
def create_dense():
    model = keras.models.Sequential()
    model.add(layers.Flatten(input_shape=(6,6,512)))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(2048, activation='elu', kernel_regularizer='l1_l2'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2048, activation='elu', kernel_regularizer='l1_l2'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    adam = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=adam,
        loss='binary_crossentropy',
        metrics=['accuracy', 'mse', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    model.summary()
    return model

In [12]:
dense_1 = create_dense()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 18432)             0         
                                                                 
 dropout_3 (Dropout)         (None, 18432)             0         
                                                                 
 dense_3 (Dense)             (None, 2048)              37750784  
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_5 (Dropout)         (None, 2048)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [ ]:
print(train_y[:20])

In [8]:
def train_model(model: tf.keras.Model, train_x, train_y, valid_x, valid_y, epochs, model_name):
    saving_models = keras.callbacks.ModelCheckpoint("./saved_models/checkpoints/" + model_name + ".{epoch:02d}.h5")
    
    return model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=epochs, batch_size=256,
                     callbacks=[saving_models])

In [9]:
valid_x = valid_x[:len(valid_x) // 2]
valid_y = valid_y[:len(valid_y) // 2]

In [13]:
for j in tqdm(range(5)):
    train_x, train_y = load_data((j*2, (j+1)*2), "train")
    for i in range(5):
        valid_x_sample = valid_x[i*len(valid_x)//5 : (i+1)*len(valid_x)//5]
        valid_y_sample = valid_y[i*len(valid_y)//5 : (i+1)*len(valid_y)//5]
        train_x_sample = train_x[i*len(train_x)//5 : (i+1)*len(train_x)//5]
        train_y_sample = train_y[i*len(train_y)//5 : (i+1)*len(train_y)//5]
        train_model(dense_1, train_x_sample, train_y_sample, valid_x_sample, valid_y_sample, 3, model_name="dense")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/3
52/52 [==============================] - 16s 284ms/step - loss: 3382.8523 - accuracy: 0.5963 - mse: 0.3772 - precision_1: 0.5824 - recall_1: 0.6011 - val_loss: 2673.3701 - val_accuracy: 0.6883 - val_mse: 0.2624 - val_precision_1: 0.6725 - val_recall_1: 0.7130
Epoch 2/3
52/52 [==============================] - 14s 264ms/step - loss: 2112.4297 - accuracy: 0.6339 - mse: 0.3209 - precision_1: 0.6207 - recall_1: 0.6356 - val_loss: 1566.8710 - val_accuracy: 0.6560 - val_mse: 0.2790 - val_precision_1: 0.7645 - val_recall_1: 0.4335
Epoch 3/3
52/52 [==============================] - 14s 263ms/step - loss: 1171.4044 - accuracy: 0.6531 - mse: 0.2862 - precision_1: 0.6433 - recall_1: 0.6436 - val_loss: 811.0563 - val_accuracy: 0.7002 - val_mse: 0.2189 - val_precision_1: 0.6926 - val_recall_1: 0.7012
Epoch 1/3
52/52 [==============================] - 14s 273ms/step - loss: 596.6624 - accuracy: 0.6163 - mse: 0.2917 - precision_1: 0.6094 - recall_1: 0.6138 - val_loss: 424.9228 - val_accurac

 20%|██        | 1/5 [03:55<15:41, 235.35s/it]

Epoch 1/3
52/52 [==============================] - 16s 303ms/step - loss: 4.8762 - accuracy: 0.5967 - mse: 0.2341 - precision_1: 0.6730 - recall_1: 0.3486 - val_loss: 4.8337 - val_accuracy: 0.6752 - val_mse: 0.2266 - val_precision_1: 0.7325 - val_recall_1: 0.5342
Epoch 2/3
52/52 [==============================] - 14s 269ms/step - loss: 4.8404 - accuracy: 0.5887 - mse: 0.2334 - precision_1: 0.6582 - recall_1: 0.3388 - val_loss: 4.8228 - val_accuracy: 0.6801 - val_mse: 0.2266 - val_precision_1: 0.7326 - val_recall_1: 0.5497
Epoch 3/3
52/52 [==============================] - 14s 271ms/step - loss: 4.8358 - accuracy: 0.5939 - mse: 0.2327 - precision_1: 0.6582 - recall_1: 0.3609 - val_loss: 4.8254 - val_accuracy: 0.6859 - val_mse: 0.2251 - val_precision_1: 0.7153 - val_recall_1: 0.5994
Epoch 1/3
52/52 [==============================] - 15s 282ms/step - loss: 4.8463 - accuracy: 0.5882 - mse: 0.2329 - precision_1: 0.6584 - recall_1: 0.3184 - val_loss: 4.8367 - val_accuracy: 0.6781 - val_mse: 

 20%|██        | 1/5 [06:36<26:25, 396.38s/it]


KeyboardInterrupt: 